In [18]:
import minsearch
import json
import openai
from openai import OpenAI
from tqdm.auto import tqdm

In [19]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [20]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [21]:
def minsearch(query):
    boost = { "question" : 3, "section" : 0.5 }

    results = index.search(
        query = query,
        filter_dict = {"course": "machine-learning-zoomcamp"},
        boost_dict = boost,
        num_results=3
    )

    return results

In [22]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. 
        Answer the QUESTION based on the CONTEXT. 
        Use only the facts from the CONTEXT when answering the QUESTION.
        If the CONTEXT doesn't contain the answer, output IDK
        Answer in at most three sentences.
        
        QUESTION: {question}
        
        CONTEXT: 
        {context}
        
    """.strip()
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [23]:
def llm(prompt):

    client = OpenAI(
        base_url='http://localhost:11434/v1/',
        api_key='ollama',
    )
      
    response = client.chat.completions.create(
        model = 'phi3',
        messages = [{"role":"user", "content":prompt}]
    )

    return response.choices[0].message.content

In [24]:
def rag(query):
    search_results = minsearch(query)
    prompt = build_prompt(query, search_results)
    return llm(prompt)

In [25]:
query = 'if the course already started, may I still join?'

In [26]:
print(rag(query))

NotFoundError: Error code: 404 - {'error': {'message': "model 'phi3' not found, try pulling it first", 'type': 'api_error', 'param': None, 'code': None}}